In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message system allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila import InMemoryConversationMemory, JSONConversationMemory
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: str) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool definition

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

In [5]:
from langrila import ToolConfig, ToolParameter, ToolProperty

tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "120", "180"],  # Gemini doesn't support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [6]:
prompt = "Turn this place into a comfortable party mood!"

### OpenAI Chat Completion

ToolConfig object is internally converted to client tool configs

In [7]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [8]:
response = chat_openai.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is set! The disco ball is spinning, energetic music is pumping at a lively 120 BPM, and the lights are dimmed to create the perfect atmosphere. Get ready to have a great time! 🎉')], name=None), usage=Usage(prompt_tokens=132, completion_tokens=45, total_tokens=177), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_XQHS6CL6ImzR1DYk6xC9B6j7', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_otsvgf7Mzwv385UqtlSd8i51', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_lZKjpvDTnHTqia2igkAK0117', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'function'}]}, {'

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! The disco ball is spinning, energetic music is pumping at a lively 120 BPM, and the lights are dimmed to create the perfect atmosphere. Get ready to have a great time! 🎉'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 132,
  'completion_tokens': 45},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_XQHS6CL6ImzR1DYk6xC9B6j7',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_otsvgf7Mzwv385UqtlSd8i51',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_lZKjpvDTnHTqia2igkAK0117',
     'function': {'arg

Asynchronous generation

In [10]:
response = await chat_openai.arun(prompt)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is now set! The disco ball is spinning, energetic music is playing at a lively beat of 120 BPM, and the lights have been dimmed to create a cozy atmosphere. Time to have some fun! 🎉💃🕺')], name=None), usage=Usage(prompt_tokens=132, completion_tokens=52, total_tokens=184), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_QfHgKYRcAC86DPVYwsdfXS3E', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_9StLc0Nm0GYkFkJOksFX7Hen', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_ozIDbDymb2YrlyUlhKZkSw04', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'function'}

multiple generation

In [11]:
response = chat_openai.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'The party mood is on! The disco ball is spinning, the energetic music is playing at a lively 120 BPM, and the lights are dimmed to create a cozy ambiance. Get ready to have a great time! 🎉💃🕺'},
  {'text': 'The party mood is set! The disco ball is spinning, the music is bumping at a lively tempo, and the lights are dimmed to create a cozy atmosphere. Let’s get the celebration started! 🕺💃🎉'}],
 'name': None}

You can get the results from specific tool.

In [12]:
response = chat_openai.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music is pumping at 120 BPM—perfect for getting everyone in the party spirit! Here are some more ways to turn the place into a comfortable party atmosphere:\n\n1. **Lighting**: Dim the lights and add some colorful LED strings or disco balls to create a fun and vibrant ambiance. Consider using soft, warm lights for a cozy feel.\n\n2. **Decorations**: Hang up some festive decorations like streamers, balloons, or themed party supplies depending on the occasion. \n\n3. **Seating Arrangement**: Create cozy seating areas with soft cushions or blankets to encourage conversation and relaxation. \n\n4. **Snack Station**: Set up a snack table with a variety of finger foods and beverages. Include options for different dietary preferences.\n\n5. **Games and Activities**: Prepare some party games or activities—like a photo booth with props, board games, or card games—to encourage interaction among guests.\n\n6. **Themed Cocktails/Mocktai

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [13]:
response = chat_openai.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=169, completion_tokens=75, total_tokens=244), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_WhSQ6nP902mcdUYgGHDGz59Y', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_VQN5OxQKPiu5TElYehE8gbVu', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_iYtAkVO4VhSkMk1mmQGFubie', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_WhSQ6nP902mcdUYgGHDGz59Y'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_V

In [14]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 169,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_WhSQ6nP902mcdUYgGHDGz59Y',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'},
  {'role': 'function',
   'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
     'call_id': 'call_VQN5OxQKPiu5TElYehE8gbVu',
     'args': '{"energetic": true, "loud": true, "bpm": "120"}',
     'funcname': 'start_music'}],
   'name': 'start_music'},
  {'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_iYtAkVO4VhSkMk1mmQGFubie',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_WhSQ6nP902mcdUYgGHDGz59Y'}

In [15]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_WhSQ6nP902mcdUYgGHDGz59Y', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_VQN5OxQKPiu5TElYehE8gbVu', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_iYtAkVO4VhSkMk1mmQGFubie', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')]

In [16]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_WhSQ6nP902mcdUYgGHDGz59Y'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_VQN5OxQKPiu5TElYehE8gbVu'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_iYtAkVO4VhSkMk1mmQGFubie')], name=None)

Azure OpenAI Chat Completion 

In [17]:
chat_openai_azure = OpenAIFunctionalChat(
    api_key_env_name="AZURE_API_KEY",  # your env variable name of OpenAI API key
    model_name="gpt-4o-2024-05-13",
    api_type="azure",
    api_version="2024-05-01-preview",
    endpoint_env_name="AZURE_ENDPOINT",
    deployment_id_env_name="AZURE_DEPLOYMENT_ID",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [18]:
response = chat_openai_azure.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The place is now set for a party! The lights are dimmed, the disco ball is spinning, and energetic music is pumping out loud with a beat of 120 BPM. Let's get this party started! 🎉💃🕺"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 133,
  'completion_tokens': 49},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_KCMrKodtSOkXOc9MfqIX7O1X',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_8apNB3OODiLaDBFb3O65LLBe',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_yJX4tHHEOuip48IQPBRG0P4a',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm":

In [19]:
response = await chat_openai_azure.arun(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The lights are dimmed to a cozy level, the disco ball is spinning, and energetic music is filling the room! The party mood is set. Enjoy! 💃🕺🎉'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 133,
  'completion_tokens': 39},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_vWxM8zZTXNnD8QQhOo5WFDKG',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_9962SbsXMch2qdN96Z2732yM',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_4r5NSFQvDHjzmZrAe0IUWey4',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
 

In [20]:
response = chat_openai_azure.run(prompt, tool_choice="start_music")

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've kicked off some energetic music! Here are some additional tips to turn this place into a comfortable party mood:\n\n1. **Lighting**: Dim the main lights and add some string lights or colorful LED lights to create a fun and cozy atmosphere.\n   \n2. **Seating**: Arrange seating areas with comfortable chairs and throw pillows. Make sure there's plenty of space for people to dance and move around.\n\n3. **Decorations**: Add some fun decorations like balloons, banners, or themed decorations that match the party vibe.\n\n4. **Snacks and Drinks**: Set up a table with a variety of snacks and beverages. Don't forget to have options for all preferences, including non-alcoholic ones.\n\n5. **Temperature**: Make sure the temperature is comfortable. If it gets hot while everyone's dancing, have a fan or an AC unit ready.\n\n6. **Games and Activities**: Plan some fun activities or games to keep everyone entertained. \n\nEnjoy t

### Gemini on Google Generative AI

You can use tools for Gemini as well which defined for OpenAI Chat Completion.

In [21]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [22]:
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, cranked up some energetic tunes, and got the disco ball spinning! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 32},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
  

Asynchronous generation

In [23]:
response = await gemini.arun(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, the music pumping, and the lights dimmed! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 29},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      

Gemini allows you to call specific tool as well as OpenAI Chat Completion.

In [24]:
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm starting the music! Let's get this party going! 🎉 🎶 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 68,
  'completion_tokens': 20},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
            string_value: "Starting music! energetic=True loud=True, bpm=\'120\'"
         

### Gemini on VertexAI

In [25]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [26]:
response = gemini_vertexai.run(prompt=prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I've got this.  Let's turn this place into a comfortable party mood! \n\nI've started some music - a nice upbeat tempo but not too loud.  The lights are dimmed to create a warm, inviting atmosphere, and the disco ball is now spinning, adding a touch of fun and sparkle. \n\nIs there anything else you want me to do?  Would you like to add some scents, adjust the temperature, or maybe even set up some games?  Just let me know! \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 73,
  'completion_tokens': 109},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: false
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
   

multiple generation

In [27]:
response = gemini_vertexai.run(prompt=prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Okay, I've started some upbeat music with a moderate tempo, dimmed the lights a bit to create a cozy vibe, and turned on the disco ball for some extra sparkle!  \n\nHow's the mood now?  Do you need anything else to get the party started?  Maybe some snacks, drinks, or a fun activity? I'm ready to help! \n"},
  {'text': "Okay, I've started some upbeat music with a moderate tempo, dimmed the lights a bit to create a cozy vibe, and turned on the disco ball for some extra sparkle!  \n\nHow's the mood now?  Do you need anything else to get the party started?  Maybe some snacks, drinks, or a fun activity? I'm ready to help! \n"}],
 'name': None}

### Anthropic Claude

tool_configs can be shared with Claude.

In [28]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [29]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4). This creates a cozy, relaxed atmosphere without making it too dark.\n2. Started playing music that is energetic but not too loud, with a moderate tempo of 120 beats per minute. This should provide a good background for conversation and light dancing if desired.\n3. Turned on the disco ball to add a fun, party-like ambiance with moving lights.\n\nThis combination should create a comfortable party mood that's lively enough for a good time but not overwhelming. The dimmed lights and moderate music volume allow for easy conversation, while the disco ball adds a festive touch.\n\nIs there anything else you'd like to adjust to make the party mood more to your liking? We can easily change the lighting, music, or disco ball settings if you want to fine-tune the atmosphere."}],
  'name': None},
 'usage': {'model_na

Asynchronous generation

In [30]:
response = await claude.arun(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights: The lights are now set to 40% brightness, creating a warm and inviting atmosphere without being too dark.\n\n2. Started the music: I've chosen energetic music to keep the party vibe going, but it's not too loud so people can still comfortably chat. The tempo is set to 120 BPM, which is a good middle ground for various dance styles and general party atmosphere.\n\n3. Activated the disco ball: The spinning disco ball will add some fun, moving lights to the space, enhancing the party feel without being overwhelming.\n\nThese settings should create a comfortable party mood that allows for both dancing and socializing. The dimmed lights and moderate music volume will make it easy for people to chat, while the energetic beats and disco ball keep the party spirit alive.\n\nIs there anything else you'd like to adjust to perfect the party

### Claude on Amazon Bedrock

In [31]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [32]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy and relaxed atmosphere without making it too dark.\n2. Started playing music that's not too energetic or loud, with a moderate tempo of 120 beats per minute. This should provide a nice background ambiance without overwhelming conversation.\n3. Turned on the disco ball to add a fun, party-like element to the room without being too intense.\n\nThese settings should create a comfortable party mood that's perfect for socializing and enjoying the atmosphere. The dimmed lights and moderate music will make it easy for people to chat and relax, while the spinning disco ball adds a touch of fun and festivity.\n\nIs there anything else you'd like me to adjust to make the party mood even more comfortable or to your liking?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-202

# Universal tool config system

Tool config the user defines is common for various llm. It is converted to client tools in clinet's FunctionCallingModule before API call. Let me show you the process of conversion.

In [33]:
from langrila.openai import OpenAIToolConfig

converted_configs = OpenAIToolConfig.from_universal_configs(tool_configs)
converted_configs

[OpenAIToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None)], required=['power']), type='function'),
 OpenAIToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None), OpenAIToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None), OpenAIToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'])], required=['energetic', 'loud', 'bpm']), type='function'),
 OpenAIToolConfig(name='dim_lights', description='Dim the lights.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToo

Then these client tool config objects is transformed to client tools by format() method.

In [34]:
converted_configs[0].format()

{'type': 'function',
 'function': {'name': 'power_disco_ball',
  'description': 'Powers the spinning disco ball.',
  'parameters': {'type': 'object',
   'properties': {'power': {'type': 'boolean',
     'description': 'Boolean to spin disco ball.'}},
   'required': ['power']}}}

Other client like Gemini and Claude is the same interface, so you could reuse tool_configs you initially defines.

In [35]:
from langrila.gemini.genai import GeminiToolConfig

In [36]:
converted_configs = GeminiToolConfig.from_universal_configs(tool_configs)
converted_configs

[GeminiToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None)], required=['power'])),
 GeminiToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None), GeminiToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None), GeminiToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'])], required=['energetic', 'loud', 'bpm'])),
 GeminiToolConfig(name='dim_lights', description='Dim the lights.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='brightness', type=

In [37]:
converted_configs[0].format()

name: "power_disco_ball"
description: "Powers the spinning disco ball."
parameters {
  type_: OBJECT
  properties {
    key: "power"
    value {
      type_: BOOLEAN
      description: "Boolean to spin disco ball."
    }
  }
  required: "power"
}

# Multi-turn conversation with tools

### For OpenAI Chat Completion

In [38]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [39]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've set the mood for a fantastic party! 🎉 The music is pumping at a lively 120 BPM, the disco ball is spinning, and the lights are dimmed for an inviting atmosphere. Get ready to have a great time! Let the festivities begin! 💃🕺"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 132,
  'completion_tokens': 57},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_OkEBWP54V1seo5AnxK1G592V',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_r80bGuTFPerkeB41ZzVNnXrZ',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_AUvyCGh92QlUx

Next turn

In [40]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've turned up the BPM to a high-energy 180! 🎶 Get ready to move and groove! The party just got even more electric! 💥💃🕺"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 250,
  'completion_tokens': 36},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_OkEBWP54V1seo5AnxK1G592V',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_r80bGuTFPerkeB41ZzVNnXrZ',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_AUvyCGh92QlUx7kSlIDgHSzV',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'func

### For Gemini

In [41]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [42]:
prompt = "Turn this place into a comfortable party mood!"

response = gemini.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, turned on the disco ball, and started some upbeat music. Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 30},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
      

In [43]:
prompt = "What is the current bpm?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The current bpm is 120.  Let me know if you'd like me to change it! 🎶 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 185,
  'completion_tokens': 25},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            b

In [44]:
prompt = "Can you make bpm up more?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've bumped up the bpm to 180.  Get those dancing shoes ready! 👟🎶 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 279,
  'completion_tokens': 25},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
        

## Multi-turn conversation using tools with multiple model

In [45]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

In [46]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The party mood is set! The disco ball is spinning, we've got energetic music pumping at 120 BPM, and the lights are dimmed for that cozy atmosphere. Let the fun begin! \U0001faa9🎶✨"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 132,
  'completion_tokens': 44},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_pgOMwZnV6HztWQ5J2CXM9JZM',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_SOt6BMgfGk8QGRcCTV2IerFU',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_7PbCZs3bNMsT8dpTvutbPHYf',
     'function': {'arguments': '{"

Claude can continue conversation without any additional process.

In [47]:
prompt = "Can you make bpm up more?"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've increased the tempo of the music to 180 BPM, which is the highest available option. This should create a much more energetic and fast-paced atmosphere for your party. The music is still set to be energetic and loud, perfect for keeping the party going strong.\n\nIs there anything else you'd like to adjust to enhance the party mood further?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 1116,
  'completion_tokens': 84},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_pgOMwZnV6HztWQ5J2CXM9JZM',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'},
    {'id': 'toolu_SOt6BMgfGk8QGRcCTV2IerFU',
     'type': 'tool_use',
     'input': {'energetic': True, 'loud': True, 'bpm': '120'},
     'name': 'start_music'},


Go back to OpenAI Chat Completion

In [48]:
prompt = "Please turn music more relax."

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music has been changed to a more relaxed vibe, set at 60 BPM. It's now softer and calmer, perfect for winding down or creating a chill party atmosphere. If there's anything else you'd like to adjust, just let me know!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 370,
  'completion_tokens': 48},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_pgOMwZnV6HztWQ5J2CXM9JZM',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_SOt6BMgfGk8QGRcCTV2IerFU',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_7PbCZs3bNMsT8dpTvutb

Gemini can follow as well

In [49]:
prompt = "Nice. So please go back to the initial bpm."

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've switched the music back to 120 BPM for you. It will maintain the relaxed vibe we have going, but with a bit more energy now.  🎶 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 578,
  'completion_tokens': 38},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
   

In [50]:
prompt = "OK, then turn brightness to 0.7."

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The lights have been adjusted to a brightness level of 0.7. This should create a slightly brighter atmosphere while still maintaining a comfortable party ambiance. The room should now feel more illuminated, which might be great for socializing or activities that require a bit more light.\n\nTo recap your current party setup:\n1. The disco ball is still spinning (from our initial setup).\n2. The music is playing at 120 BPM, not too energetic or loud.\n3. The lights are now at 70% brightness.\n\nIs there anything else you'd like to change or adjust to perfect your party atmosphere?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 1677,
  'completion_tokens': 137},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_pgOMwZnV6HztWQ5J2CXM9JZM',
     'type': 'tool

In [51]:
prompt = "Finally stop the disco ball."

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The disco ball has been turned off. Your party atmosphere is now more relaxed and refined without the spinning lights. If there's anything else you'd like to adjust or any other way I can assist, just let me know! Enjoy the party! 🎉"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 743,
  'completion_tokens': 49},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_pgOMwZnV6HztWQ5J2CXM9JZM',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_SOt6BMgfGk8QGRcCTV2IerFU',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_7PbCZs3bN

If you input non-relavant topic, any tool is not called.

In [52]:
prompt = "What is the weather today?"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I'm unable to provide real-time information like today's weather. However, you can easily check a reliable weather website or smartphone app for the most accurate and up-to-date weather information in your area. If you have any other questions or need assistance with planning based on the weather, feel free to ask!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 806,
  'completion_tokens': 59},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_pgOMwZnV6HztWQ5J2CXM9JZM',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_SOt6BMgfGk8QGRcCTV2IerFU',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}'

Conversation is recorded with serialized universal message format.

In [60]:
shared_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_pgOMwZnV6HztWQ5J2CXM9JZM'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_SOt6BMgfGk8QGRcCTV2IerFU'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_7PbCZs3bNMsT8dpTvutbPHYf'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_pgOMwZnV6HztWQ5J2CXM9JZM',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_SOt6BMgfGk8QGRcCTV2IerFU',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_7PbCZs3

# Multi-turn conversation with tools and context truncation

Although context truncation doesn't work accurately if we use tools, but truncation is useful for token management to some extent.

In [53]:
from langrila import InMemoryConversationMemory

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    context_length=50,
)

In [54]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt, tool_choice="auto")

In [55]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt, tool_choice="auto")

Input message is truncated because total length of messages exceeds context length.
Input message is truncated because total length of messages exceeds context length.


User message and assistant message are target of truncation while tool call message and tool result message is not.

In [56]:
print(response.message.content[0].text)

The music is now pumping at 180 BPM! Get ready to dance even harder! 🎶💥


In [57]:
response.usage

Usage(prompt_tokens=209, completion_tokens=21, total_tokens=230)

In [58]:
response.prompt

[{'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_Ohj1uawqp59mRcRSrlArypeQ',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_7ELiVM0gWDmeveUB30H0WOyx',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_EAm1adap4wLVVCG6uEf3jpeI',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_Ohj1uawqp59mRcRSrlArypeQ',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_7ELiVM0gWDmeveUB30H0WOyx',
  'name': 'start_music',
  'content': "Starting music! energetic=True loud=True, bpm='120'"},
 {'role': 'tool',
  'tool_call_id': 'call_EAm1adap4wLVVCG6uEf3jpeI',
  'name': 'dim_lights',
  'content': 'Lights are now set to 0.5'},
 {'role': 

In [59]:
# whole history
chat_openai.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_Ohj1uawqp59mRcRSrlArypeQ'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_7ELiVM0gWDmeveUB30H0WOyx'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_EAm1adap4wLVVCG6uEf3jpeI'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_Ohj1uawqp59mRcRSrlArypeQ',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_7ELiVM0gWDmeveUB30H0WOyx',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_EAm1ada